Now we will use a pre-trained model (distilbert) for sentiment analysis, we will start with installing necessary components.

In [1]:
pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

**1. Loading our Dataset 'imdb':**

We will also import the same 'imdb' dataset for this run.

In [4]:
from datasets import load_dataset

In [5]:
imdb = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [23]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

**2. Preprocessing:**

We will load distilbert tokenizer to preprocess the text.

In [7]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Create a function that will tokenize and truncate according to distilbert requirements.

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Now we will apply the function to our 'imdb' dataset.

In [10]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [25]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [27]:
print(len(tokenized_imdb["train"]))  # Replace with the correct split name
print(training_args.per_device_train_batch_size)


25000
16


Now we will pad the sentences to the maximum length for all texts.

In [11]:
from transformers import DataCollatorWithPadding

In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**3. Evaluation:**

We will include the accuracy metric during training to evaluate our model's performance.

In [13]:
import evaluate

In [14]:
accuracy = evaluate.load("accuracy")

We will create a function that will pass our results and evaluate them.

In [15]:
import numpy as np

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

**5. Training the model:**

We will create a map of the expected ids to their labels with id2label and label2id

In [17]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

Prepare for training.

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define our training hyperparameters.

In [20]:
training_args = TrainingArguments(
    output_dir="sentiment_analysis_model",
    run_name="trial_run",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Now we can train.

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.221500,0.207077,0.921280
2,0.145500,0.236552,0.931040


TrainOutput(global_step=3126, training_loss=0.20413226800626924, metrics={'train_runtime': 3012.041, 'train_samples_per_second': 16.6, 'train_steps_per_second': 1.038, 'total_flos': 6556904415524352.0, 'train_loss': 0.20413226800626924, 'epoch': 2.0})

Push the model to huggingface

In [37]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/tashrifmahmud/sentiment_analysis_model/commit/c16c89a8fea2c7015d1d134b58d4e850a8ba8f28', commit_message='End of training', commit_description='', oid='c16c89a8fea2c7015d1d134b58d4e850a8ba8f28', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tashrifmahmud/sentiment_analysis_model', endpoint='https://huggingface.co', repo_type='model', repo_id='tashrifmahmud/sentiment_analysis_model'), pr_revision=None, pr_num=None)

Evaluation:

In [48]:
# Evaluate the model
results = trainer.evaluate()

print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.20707689225673676, 'eval_accuracy': 0.92128, 'eval_runtime': 354.7526, 'eval_samples_per_second': 70.472, 'eval_steps_per_second': 4.406, 'epoch': 2.0}


Saving files for now.

In [39]:
!zip -r sentiment_analysis_model.zip sentiment_analysis_model

  adding: sentiment_analysis_model/ (stored 0%)
  adding: sentiment_analysis_model/checkpoint-3126/ (stored 0%)
  adding: sentiment_analysis_model/checkpoint-3126/tokenizer_config.json (deflated 76%)
  adding: sentiment_analysis_model/checkpoint-3126/training_args.bin (deflated 51%)
  adding: sentiment_analysis_model/checkpoint-3126/trainer_state.json (deflated 65%)
  adding: sentiment_analysis_model/checkpoint-3126/rng_state.pth (deflated 25%)
  adding: sentiment_analysis_model/checkpoint-3126/config.json (deflated 47%)
  adding: sentiment_analysis_model/checkpoint-3126/special_tokens_map.json (deflated 42%)
  adding: sentiment_analysis_model/checkpoint-3126/scheduler.pt (deflated 56%)
  adding: sentiment_analysis_model/checkpoint-3126/optimizer.pt (deflated 14%)
  adding: sentiment_analysis_model/checkpoint-3126/model.safetensors (deflated 8%)
  adding: sentiment_analysis_model/checkpoint-3126/vocab.txt (deflated 53%)
  adding: sentiment_analysis_model/checkpoint-3126/tokenizer.json 

In [40]:
from google.colab import files

files.download("sentiment_analysis_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
!zip -r wandb.zip wandb

  adding: wandb/ (stored 0%)
  adding: wandb/latest-run/ (stored 0%)
  adding: wandb/latest-run/files/ (stored 0%)
  adding: wandb/latest-run/files/requirements.txt (deflated 55%)
  adding: wandb/latest-run/files/wandb-metadata.json (deflated 47%)
  adding: wandb/latest-run/files/output.log (deflated 83%)
  adding: wandb/latest-run/logs/ (stored 0%)
  adding: wandb/latest-run/logs/debug-core.log (deflated 58%)
  adding: wandb/latest-run/logs/debug-internal.log (deflated 83%)
  adding: wandb/latest-run/logs/debug.log (deflated 83%)
  adding: wandb/latest-run/tmp/ (stored 0%)
  adding: wandb/latest-run/tmp/code/ (stored 0%)
  adding: wandb/latest-run/run-w5o2brfc.wandb (deflated 80%)
  adding: wandb/debug-internal.log (deflated 83%)
  adding: wandb/debug.log (deflated 83%)
  adding: wandb/run-20241124_020637-w5o2brfc/ (stored 0%)
  adding: wandb/run-20241124_020637-w5o2brfc/files/ (stored 0%)
  adding: wandb/run-20241124_020637-w5o2brfc/files/requirements.txt (deflated 55%)
  adding: wan

In [42]:
files.download("wandb.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Testing how the model performs on unseen text.

In [43]:
from transformers import pipeline

pipe = pipeline("text-classification", model="tashrifmahmud/sentiment_analysis_model")

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [45]:
sample_text_1 = "Oh my god, the actor is so amazing here."

pipe(sample_text_1)

[{'label': 'POSITIVE', 'score': 0.9676467180252075}]

In [46]:
sample_text_2 = "I would've been able to understand everything in this complicated story - if I could've HEARD half of the dialogue. I watched parts of this film in three different environments, and even in the best one way too many important lines were simply unintelligble. Since this is apparently a conscious choice by the director, I doubt I'll ever watch another Nolan film."

pipe(sample_text_2)


[{'label': 'NEGATIVE', 'score': 0.9698693752288818}]

In [47]:
sample_text_3 = "Some people can say this is too complicated moive and there is no feeling at all, after watching this. But didn't watch any moive that ends connection between two characters like this. So it is the best end I ever watched. Remember the how Vin diesel and Paul walker end their relationship by looking at each other and go different paths at a road junction. In this film it would be like Protagonist and neil meets at a same road in different direction, and neils know this is the end of their relationship and Protagonist knows he will again meet him at some distance.... Not only the end, the plot is superb. It's like I watch a half of a movie with one half my brain and other part of my brain knows what is going to happen in the other pat. In deeper sense..Neil and Protagonist is not the different characters. They are the reflections of a same person. The best moive I ever watched. Thank you Christopher Nolan.. I would love to see a series based on the world of Tenet."

pipe(sample_text_3)


[{'label': 'POSITIVE', 'score': 0.9903952479362488}]